In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 7)

# 📊 Полный анализ Two-Stage Detection

In [ ]:
# Загружаем ВСЕ результаты
all_results = pd.read_csv("../docs/two_stage_detection_results.csv")
ml_results = pd.read_csv("../docs/ml_checked_transactions.csv")

print(f"📊 Total transactions: {len(all_results)}")
print(f"📊 ML-checked: {len(ml_results)}")
print(f"📊 Auto-approved: {len(all_results) - len(ml_results)}")

# ВАЖНО! Проверяем сколько ВСЕГО фродеров
if 'target' in all_results.columns:
    total_frauds = all_results['target'].sum()
    ml_checked_frauds = ml_results['target'].sum() if 'target' in ml_results.columns else 0
    auto_approved_frauds = total_frauds - ml_checked_frauds
    
    print(f"\n🎯 FRAUD DISTRIBUTION:")
    print(f"   Total frauds in dataset: {total_frauds}")
    print(f"   Sent to ML check: {ml_checked_frauds}")
    print(f"   ❌ Auto-approved (MISSED by scorecard): {auto_approved_frauds}")
    print(f"\n   📊 Scorecard Recall: {ml_checked_frauds/total_frauds*100:.1f}%")
    print(f"      (Из {total_frauds} фродов отправил {ml_checked_frauds} на ML-проверку)")

all_results.head()

## Анализ ML-модели (на тех кто прошел scorecard)

In [ ]:
if 'target' in ml_results.columns and 'fraud_prediction' in ml_results.columns:
    from sklearn.metrics import confusion_matrix, classification_report
    
    actual = ml_results['target'].sum()
    predicted = ml_results['fraud_prediction'].sum()
    
    print(f"\n🤖 ML MODEL RESULTS (on {len(ml_results)} checked transactions):")
    print(f"   Actual frauds: {actual}")
    print(f"   Predicted frauds: {predicted}")
    print(f"   Difference: {predicted - actual}")
    
    cm = confusion_matrix(ml_results['target'], ml_results['fraud_prediction'])
    print(f"\n   Confusion Matrix:")
    print(f"   TN: {cm[0,0]} | FP: {cm[0,1]}")
    print(f"   FN: {cm[1,0]} | TP: {cm[1,1]}")
    
    print(f"\n   💡 Объяснение:")
    print(f"   Predicted ({predicted}) = TP ({cm[1,1]}) + FP ({cm[0,1]})")
    print(f"   TP = правильно нашли фрод")
    print(f"   FP = ошибочно назвали фродом (false alarm)")
    print(f"   FN = пропустили фрод")
    
    print("\n   Classification Report:")
    print(classification_report(ml_results['target'], ml_results['fraud_prediction'], 
                                target_names=['Non-Fraud', 'Fraud']))

## ИТОГОВАЯ статистика по ВСЕМ фродам

In [ ]:
if 'target' in all_results.columns:
    total_frauds = all_results['target'].sum()
    
    # Сколько поймали
    if 'fraud_prediction' in all_results.columns:
        detected = all_results['fraud_prediction'].sum()
        
        # Реально правильно определенные
        true_detected = ((all_results['target'] == 1) & (all_results['fraud_prediction'] == 1)).sum()
        
        print("\n" + "="*60)
        print("🎯 FINAL SUMMARY (entire pipeline)")
        print("="*60)
        print(f"Total frauds in dataset: {total_frauds}")
        print(f"Frauds detected: {true_detected}")
        print(f"Frauds missed: {total_frauds - true_detected}")
        print(f"\nOverall Recall: {true_detected/total_frauds*100:.1f}%")
        print(f"Overall Precision: {true_detected/detected*100:.1f}%" if detected > 0 else "N/A")
        print("="*60)
        
        # Детали пропущенных
        missed = all_results[(all_results['target'] == 1) & (all_results['fraud_prediction'] == 0)]
        if len(missed) > 0:
            print(f"\n❌ MISSED FRAUDS ({len(missed)}):")
            if 'detection_stage' in missed.columns:
                print(missed['detection_stage'].value_counts())
            if 'scorecard_total' in missed.columns:
                print(f"\nTheir scorecard scores:")
                print(missed['scorecard_total'].value_counts().sort_index())

## False Positives Analysis

In [ ]:
if 'target' in all_results.columns and 'fraud_prediction' in all_results.columns:
    fp = all_results[(all_results['target'] == 0) & (all_results['fraud_prediction'] == 1)]
    
    print(f"\n🔍 FALSE POSITIVES: {len(fp)}")
    
    if len(fp) > 0 and 'fraud_probability' in fp.columns:
        print(f"   Mean probability: {fp['fraud_probability'].mean():.3f}")
        
        if 'scorecard_total' in fp.columns:
            print(f"\n   Scorecard distribution:")
            print(fp['scorecard_total'].value_counts().sort_index())
        
        print(f"\n   Top 5 examples:")
        display_cols = ['fraud_probability', 'scorecard_total', 'rare_os_flag', 'rare_device_flag']
        display_cols = [c for c in display_cols if c in fp.columns]
        print(fp[display_cols].head())